## Colectiva viz
Este notebook tiene código para generar visualizaciones de un conjunto de archivos con lecturas de EEG.

In [ ]:
%%capture
!pip install mne

In [ ]:
#https://github.com/curiositry/EEGrunt/blob/master/data/eegrunt-obci-ovibe-test-data.csv

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import mne

# esto lee una grabacion que hice de muestra, pensé que eran 5min pero creo que es menos
# df_fnames = [
#        "/content/alew_EEG_2022-04-28_200016.csv",
#        "/content/amigamile_EEG_2022-04-29_190300.csv",
#        "/content/erik_EEG_2022-04-30_185308.csv",
#        "/content/fer_EEG_2022-04-29_183739.csv",
#        "/content/guiada1_EEG_2022-04-28_194425.csv",
#        "/content/quetzalli_EEG_2022-04-30_134611.csv",
# ]

df_fnames = [
       "/content/alf_audio_EEG_2022-04-20_155442.csv",
       "/content/emme_audio_EEG_2022-04-28_145148.csv",
       "/content/feli_audio_EEG_2022-04-28_174706.csv",
       "/content/les_audio_EEG_2022-04-20_153820.csv",
]

dfs = []
for fname in df_fnames:
  df = pd.read_csv(fname, header=0, index_col = False)
  dfs.append(df)

In [ ]:
for df in dfs:
  print(len(df))

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(30,30))
for i, ax in enumerate(fig.axes):
    ax.plot(dfs[i].iloc[-38811:,1:])
    # ax.plot(data_fft[i][1:])
    ax.title.set_text(df_fnames[i].split('/')[2].split('_')[0])
    ax.set_xlabel('Time')
    ax.set_ylabel('Amplitude')

    #ax.set_xticks([])
    #ax.set_yticks([])
plt.show()

In [ ]:
def do_fft(all_channel_data):
    data_fft = list(map(lambda x: np.fft.fft(x),all_channel_data))
    return data_fft
    
def get_frequency(all_channel_data): 
    L = len(all_channel_data[0])
    Fs = 64

    #data_fft =  map(lambda x: np.fft.fft(x), all_channel_data)

    data_fft = do_fft(all_channel_data)

    #Compute frequency
    freq1 = list(map(lambda x: abs((x/L)),data_fft))
    frequency = list(map(lambda x: x[:int((L/2+1))] * 2, freq1))
    
    #List frequency
    delta = list(map(lambda x: x[int(L*1//Fs-1):int(L*4/Fs)],frequency))
    #abc = map(lambda x: int(x), list(delta))
    #print("> " + str(abc))
    theta = list(map(lambda x: x[int(L*4//Fs-1):int(L*8//Fs)],frequency))
    alpha = list(map(lambda x: x[int(L*5//Fs-1):int(L*13//Fs)],frequency))
    beta = list(map(lambda x: x[int(L*13//Fs-1):int(L*30//Fs)],frequency))
    gamma = list(map(lambda x: x[int(L*30//Fs-1):int(L*50//Fs)],frequency))
   
    # import pdb;pdb.set_trace()
    return delta,theta,alpha,beta,gamma

def get_feature(all_channel_data): 
		"""
		Get feature from each frequency.
		Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
		Output: Feature (standard deviasion and mean) from all frequency bands and channels with dimesion 1 x M (number of feature).
		"""
		#Get frequency data
		(delta,theta,alpha,beta,gamma) = get_frequency(all_channel_data)

		#Compute feature std
		delta_std = np.std(delta, axis=1)
		theta_std = np.std(theta, axis=1)
		alpha_std = np.std(alpha, axis=1)
		beta_std = np.std(beta, axis=1)
		gamma_std = np.std(gamma, axis=1)

		#Compute feature mean
		delta_m = np.mean(delta, axis=1)
		theta_m = np.mean(theta, axis=1)
		alpha_m = np.mean(alpha, axis=1)
		beta_m = np.mean(beta, axis=1)
		gamma_m = np.mean(gamma, axis=1)

		#Concate feature
		# feature = np.array([delta_std,delta_m,theta_std,theta_m,alpha_std,alpha_m,beta_std,beta_m,gamma_std,gamma_m])
		feature = np.array([delta_m,theta_m,alpha_m,beta_m,gamma_m])
		# feature = feature.T
		feature = feature.ravel()
  
		return feature

def compute_rolling_features(df_, size = 64):
    rolling_feat = {'delta_m':[],
                    'theta_m':[],
                    'alpha_m':[],
                    'beta_m':[],
                    'gamma_m':[]}

    for i in range(size, len(df_)+1):
        feat = get_feature(np.asarray(df.T.iloc[:,i-size:i]))
        rolling_feat['delta_m'].append(feat[0:5])
        rolling_feat['theta_m'].append(feat[5:10])
        rolling_feat['alpha_m'].append(feat[10:15])
        rolling_feat['beta_m'].append(feat[15:20])
        rolling_feat['gamma_m'].append(feat[20:25])

    return rolling_feat


In [ ]:
rolling_waves_mu = []

waves = ['delta_m', 'theta_m', 'alpha_m', 'beta_m', 'gamma_m']
fig, axs = plt.subplots(len(dfs), 1, figsize=(30,30))
for i, ax in enumerate(fig.axes):
    test = compute_rolling_features(dfs[i])
    ax.plot(test)
    ax.title.set_text(df_fnames[i].split('/')[2].split('_')[0])
    # ax.legend()


    #ax.set_xticks([])
    #ax.set_yticks([])
plt.show()